# Forest Fire Model

Make it possible to set the probability parameters p, f, q. Start with an empty field (all cells
are in state Ashes).
Write the number of cells for each state (Ashes, Tree, Fire) for each time step into a file
(three values per line, ASCII format, separated by blanks).

q - rate of induced growth\
f - rate of spontaneous fire\
p - rate of spontaneous growth

In [21]:
import numpy as np
import time
import os
from IPython.display import clear_output

In [22]:
q = 0.1 #1 - induced growth. 
f = 0.01
p = 0.001
nx, ny = (101, 82)
iterations = 50
neighbourhood = ((-1,0), (0,-1), (0, 1), (1,0)) #von neumann neighbourhood

### Auxilary functions

In [23]:
def count_cells(forest):
    ashes_count = np.count_nonzero(forest == 0)
    tree_count = np.count_nonzero(forest == 1)
    fire_count = np.count_nonzero(forest == 2)
    return ashes_count, tree_count, fire_count

In [24]:
def color_matrix(matrix):
    color_map = {"0": "\033[0;33m", "1": "\033[0;32m", "2": "\033[0;31m"}
    colored_str = ""
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            value = str(matrix[i][j])
            if value in color_map:
                colored_str += color_map[value] + value + "\033[0m"
            else:
                colored_str += value
            colored_str += ""
        colored_str += "\n"
    print(colored_str)  

In [25]:
print(color_matrix(np.array([[1,0,1],
                      [0,2,1]]),
                     ))

101
021

None


In [26]:
def update_forest(X):
    """One iteration of forest-fire CA"""
    X1 = np.zeros(shape=X.shape, dtype=np.uint64)
    nx = X.shape[1]
    ny = X.shape[0]
    for ix in range(0,nx):
        for iy in range(0,ny):

            if X[iy,ix] == 0 and np.random.random() <= p: #growth of a random tree
                X1[iy,ix] = 1
            elif X[iy,ix] == 0:
                for dx,dy in neighbourhood:
                    if X[(iy+dy)%ny,(ix+dx)%nx] == 1 and np.random.random() < q: #induced growth rate
                        #(iy+dy)%ny ensures torus topology. Thanks to that cell with id 0 is a neighbour with cell with id ny.
                        X1[iy,ix] = 1
                        break

            if X[iy,ix] == 1:
                X1[iy,ix] = 1
                for dx,dy in neighbourhood:
                    if X[(iy+dy)%ny,(ix+dx)%nx] == 2:
                        X1[iy,ix] = 2
                        break
                else:
                    if np.random.random() <= f:
                        X1[iy,ix] = 2
    return X1


Let 0 be ashes (yellow), 1 - forest (green) and 2 - fire (red)

In [27]:
forest  = np.zeros((ny, nx), dtype=np.uint64)

In [ ]:
print("Current parameters: \n")
print(f"width: {nx}, height = {ny} \n")
print(f"induced growth rate q = {q} \n")
print(f"spontaneous tree growth rate q = {p} \n")
print(f"induced fire rate q = {f} \n")
print(f"Number of iterations : {iterations} \n")   

In [28]:
out_file = open("forest_fire_output.txt", "w")
ashes_count, tree_count, fire_count = count_cells(forest)
out_file.write(f"{ashes_count} {tree_count} {fire_count}\n") # initial number of different cells. 100% of cells should be ashes at the beginning
# Run the forest fire automaton for 50 time steps and write the cell counts to the output file
for _ in range(iterations):
    clear_output(wait=True)
    forest = update_forest(forest)
    ashes_count, tree_count, fire_count = count_cells(forest)
    out_file.write(f"{ashes_count} {tree_count} {fire_count}\n")
    color_matrix(forest)
    time.sleep(0.5)

# Close the output file
out_file.close()

20220210002000101001022110220112111111010111200002100111101111110110202000002121112110210020102201101
12121001121102111220010120112100110201001011221101021120221120200202000001000102100002111002012011100
21100021201021111112001110021111110011101002022122002011111000201011211101112021100011110112020012202
00110011120211110011220210221120200021010021100001000211110200111111012111010011111112110020102000201
02202110102111111011112120000111111000201211022002101011111110202020100200000101121100201000100200122
21100111022111100101111201110021120102101111222011201111120010121101212112010200200202120102112220012
21211020220211111101112011202202100111121021111102000111201110212012011111000022110121201101120020120
01011210000021111202121201110200001021100100101021111212101112110111210000201121111112121111201101111
01211011001111100000002011100010121211102110000011002020211110111021100110000201012000212022100011110
0010022011202112011002000202201111011101120211011120001102002001021211021111200101